In [1]:
import numpy as np
from sklearn.linear_model import QuantileRegressor

In [77]:
X = np.c_[np.ones(100), np.random.normal(0, 5, size=(100, 5))]
beta = np.array([10, 1, 2, 3, 4, 5])

In [78]:
y = X @ beta + np.random.normal(0, 1, 100)

In [114]:
class QuantileRegression:
    def __init__(self, theta, verbose=False):
        self.theta = theta
        self.verbose = verbose
        
    def fit(self, X, y, learning_rate=0.001, num_iter=1000):
        self.dim = X.shape[1]
        self.num_iter = num_iter
        self.learning_rate = learning_rate
        self.params = np.random.uniform(0, 1, self.dim)
        self.gradient_descent(X, y)
    
    def sum_abs_loss(self, y, y_hat):
        dev = y - y_hat
        return np.sum(np.maximum(self.theta * dev, (self.theta - 1) * dev))
    
    def gradient_descent(self, X, y):
        self.count = 0
        y_hat = X @ self.params
        best_loss = self.sum_abs_loss(y, y_hat)
        
        if self.verbose:
            print(f'{self.count}/{self.num_iter}:', best_loss)
        
        self.best_params = np.copy(self.params)

        for i in range(self.num_iter):
            # update params
            grads = self.gradient(X, y, self.params)
            self.params -= grads * self.learning_rate
            
            # print loss
            y_hat = X @ self.params
            self.count += 1
            loss = self.sum_abs_loss(y, y_hat)
            
            if self.verbose:
                print(f'{self.count}/{self.num_iter}:', loss)
            
            if loss < best_loss:
                self.best_params = np.copy(self.params)
                best_loss = loss

    def gradient(self, X, y, params):
        y_hat = X @ self.params
        dev = y - y_hat
        grads = np.zeros(self.dim)
        
        # this gradient for sum(|y - xb|) only
        for i in range(len(grads)):
            grads[i] = np.sum(np.where(dev > 0,
                                       - self.theta * np.sign(X[:, i] * self.params[i]),
                                       (1 - self.theta) * np.sign(X[:, i] * self.params[i])))
        return grads

In [115]:
for q in [0.01, 0.05, 0.2, 0.5, 0.7, 0.95, 0.99]:
    qr = QuantileRegression(q)
    qr.fit(X, y)
    print(qr.params)
    
    qr_sklearn = QuantileRegressor(quantile=q, alpha=0, fit_intercept=False)
    qr_sklearn.fit(X,y)
    print(qr_sklearn.coef_)

[-0.64708373  1.00918346  1.71656298  3.08060967  3.48084359  4.32914151]
[7.84778997 0.95973058 2.06227284 2.96423484 4.02349655 4.94364497]
[3.35677069 1.00386861 2.05566202 3.11606055 3.84708736 4.41111839]
[8.55834054 0.99959593 1.99106566 2.97300592 4.03515452 5.0014718 ]
[-16.49892749   1.08413108   2.15388939   2.76508997   4.07227252
   3.76337485]
[9.41706562 1.01233848 2.01723498 3.000608   3.96325221 5.04944315]
[10.15348854  1.00506953  1.97174569  3.02026303  3.99561627  5.07963791]
[10.12990076  0.99071535  1.96491419  3.02241105  3.99358358  5.06107899]
[10.67124141  1.02889153  2.03328617  3.03575993  3.9795603   5.09724803]
[10.5818026   1.00718279  1.99175597  3.03488802  3.99034049  5.0387914 ]
[11.91995448  1.02421135  2.11585989  3.00662923  3.95939441  5.11728873]
[11.66465655  1.06820416  2.02541937  3.05573165  4.00386565  5.0026522 ]
[12.24137519  1.06615256  2.11507001  2.99171828  3.97967561  5.11776363]
[12.11976851  0.98919757  1.97207873  3.06047571  3.987